In [1]:
import gzip
from src.distance_metrics.ncd_metric import calculate_ncd
from src.algorithms.knn_compressor import KnnCompressor
from src.algorithms.ce_compressor import CeCompressor

In [2]:
cx1 = 10
cx2 = 13
cx1cx2 = 20

In [3]:
calculate_ncd(cx1=cx1, cx2=cx2, cx1cx2=cx1cx2)
# Ncd.calculate(cx1=cx1, cx2=cx2, cx1cx2=cx1cx2)

0.7692307692307693

In [4]:
text = 'Hello, I am coding a quick streamlit application'
text2 = 'Streamlit is a good python application'
text3 = 'I like Arsenal football team'

In [5]:
cx1 = len(gzip.compress(text.encode()))
cx2 = len(gzip.compress(text2.encode()))
cx3 = len(gzip.compress(text3.encode()))

# x1x2 = " ".join([text, text2])
# x1x3 = " ".join([text, text3])
# x2x3 = " ".join([text2, text3])

x1x2 = " ".join([text2, text])
x1x3 = " ".join([text3, text])
x2x3 = " ".join([text3, text2])

cx1x2 = len(gzip.compress(x1x2.encode()))
cx1x3 = len(gzip.compress(x1x3.encode()))
cx2x3 = len(gzip.compress(x2x3.encode()))


print(cx1, cx2, cx3)

68 58 48


In [6]:
calculate_ncd(cx1=cx1, cx2=cx2, cx1cx2=cx1x2)

0.45588235294117646

In [7]:
calculate_ncd(cx1=cx1, cx2=cx3, cx1cx2=cx1x3)

0.5588235294117647

In [8]:
calculate_ncd(cx1=cx2, cx2=cx3, cx1cx2=cx2x3)

0.5517241379310345

In [8]:
import bz2
import lzma 
# import zstd

# len(bz2.compress(x1x2.encode()))

In [10]:
compressor = lzma 
len(compressor.compress(x1x2.encode()))

132

In [6]:
import pandas as pd 
import numpy as np 
train = pd.read_csv('./src/data/train.csv')
test = pd.read_csv('./src/data/test.csv')

In [3]:
train = train.sample(frac=0.01)
test = test.sample(frac=0.01)

In [3]:
test_df = test.groupby('Class Index').apply(lambda x: x.sample(frac=0.01)).reset_index(drop = True)
test_df.groupby('Class Index').size()

Class Index
1    19
2    19
3    19
4    19
dtype: int64

In [11]:
ts_list = [d for _, d in test_df.groupby('Class Index')]
full_txt_list = [' '.join(df["Description"]) for df in ts_list]
len(full_txt_list)

4

'Nigerian rebel leader Mujahid Dokubo-Asari said he arrived in the capital Abuja on Wednesday for talks with President Olusegun Obasanjo after his group threatened a new offensive against troops in the oil-producing Niger delta and Suspected Maoist rebels kidnap a judge and a former adviser to the late king, Nepalese police say.Pakistani troops in Waziristan begin a new search for Islamic militants, among them a former US detainee.KATHMANDU: Nepali troops killed at least 21 Maoist rebels and lost 13 soldiers in gunbattles across the Himalayan kingdom, officials said yesterday.Canadian Press - THE HAGUE, Netherlands (AP) - A bombing before dawn Monday blew the front door off a Muslim elementary school in a southern town and extensively damaged the building in what police suspect was a revenge attack for the killing of a Dutch filmmaker last week.Reuters - Drunken revelers at the world\'s\\biggest beer festival this year can purchase a stiff linguistic\\pick me-up for when their brains g

In [4]:
train.shape

(1200, 3)

In [5]:
test.shape

(76, 3)

In [4]:
# train_arrays = np.array(train[['Description', 'Class Index']])
test_arrays = np.array(test[['Description', 'Class Index']])
# to_predict_arrays = np.array(test[['Description', 'Class Index']])

In [5]:
# algo = KnnCompressor(compressor=gzip,
#                      base_df=train,
#                      to_predict_df=test)

algo = CeCompressor(base_df=train,
                    to_predict_df=test)

In [6]:
algo.run(text_col='Description', target_col='Class Index')

In [7]:
from sklearn.metrics import f1_score

In [8]:
true = test_arrays[:, 1]

In [9]:
# pred = algo.final_predict_result
pred = algo.result_df['prediction']

In [10]:
f1_score(list(true), pred, average='weighted')

0.11964188822571893

In [13]:
import multiprocessing as mp
import itertools
cpu_count = mp.cpu_count()
# pool = mp.Pool(mp.cpu_count())
# some function

# pool.close()
cpu_count

10

In [14]:
# a = ['foo', 'bar', 'baz']
# b = ['x', 'y', 'z', 'w']

a = test['Description'].values.tolist()
b = train['Description'].values.tolist()

text_list = [x for x in itertools.product(a, b)]

In [19]:
len(a)

76

In [29]:
lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]

chunk = lol(distance, len(b))
len(chunk)


76

In [30]:
len(chunk[0])

1200

In [47]:
from statistics import mode

test1 = chunk[2]
sorted_idx = np.argsort(np.array(test1))
top_k_class = train.reset_index(drop=True).loc[sorted_idx[:5], 'Class Index']
mode(top_k_class)

2

In [49]:
# FINAL algorithm w/ multiprocessing 

# need to define target and text column

text_col = '...'
target_col = 'Class Index'

a = test['Description'].values.tolist()
b = train['Description'].values.tolist()

text_list = [x for x in itertools.product(a, b)]

preds = []
chunking = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
distance = []
pool = mp.Pool(mp.cpu_count())
for d in pool.starmap(calculate_ncd_mp, text_list, chunksize=5):
    distance.append(d)
pool.close()

chunks = chunking(distance, len(b))
for i in range(len(chunks)):
    sorted_idx = np.argsort(np.array(chunks[i]))
    top_k_class = train.reset_index(drop=True).loc[sorted_idx[:5], target_col]
    preds.append(mode(top_k_class))


In [16]:
len(distance)

91200

In [13]:
df_split = np.array_split(train, cpu_count)
len(df_split)

10